In [ ]:
import torch
import monai
import dataloader
import os
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split


In [ ]:
data_path = '../data/hm30rad/'
df = pd.read_csv('../data/train_labels.csv')
names = df['tomo_id'].astype(str).unique().tolist()

In [ ]:
train_names, val_names = train_test_split(names, test_size=0.2)

train_dataset = dataloader.MMapDataset(names=train_names, path=data_path, gpu=True)
val_dataset = dataloader.MMapDataset(names=val_names, path=data_path, gpu=True)

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=dataloader.custom_collate, shuffle=True) # can put data aug in collate func later to have optional aug
val_loader = DataLoader(val_dataset, batch_size=16, collate_fn=dataloader.custom_collate, shuffle=False)

In [ ]:
# model = YourModel().to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(10):
    # model.train()
    i=0
    for batch in dataloader:
        src, tgt = batch['src'].to(device), batch['tgt'].to(device)
        print(f'batch {i} done')
        i += 1
        # optimizer.zero_grad()
        # outputs = model(inputs)
        # loss = criterion(outputs, targets)
        
        # loss.backward()
        # optimizer.step()
        
        # print(f"Epoch {epoch}, Loss: {loss.item()}")
    
    # model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for batch in dataloader:
            inputs, targets = batch['src'].to(device), batch['tgt'].to(device)
        #     outputs = model(inputs)
        #     loss = criterion(outputs, targets)
        #     val_loss += loss.item()
        # print(f"Validation Loss: {val_loss / len(dataloader)}")